In [1]:
#!pip install psycopg2

In [2]:
import pandas as pd
from sqlalchemy import create_engine

In [3]:
#csv_file = "test.csv"
#df = pd.read.csv(csv_file)
#df.head()

In [29]:
#Avocado In
#Modified CSV before reading for formatting
Avocado_file = "avo_prices.csv"
Avocado_df = pd.read_csv(Avocado_file)
Avocado_df = Avocado_df[["New Date", "AveragePrice", "Total Volume", "Small Bags", "XLarge Bags", "type", "year", "region"]]
Avocado_df = Avocado_df.rename(columns={"New Date": "date", "AveragePrice": "average_price", "Total Volume": "total_volume", "Small Bags": "small_avocados_sold", "XLarge Bags": "xl_avocados_sold"})
Avocado_df.head()

,date,average_price,total_volume,small_avocados_sold,xl_avocados_sold,type,year,region
0,1/5/2015,1.22,40873.28,9186.93,0.0,conventional,2015,Albany
1,1/5/2015,1.00,435021.49,16707.15,0.0,conventional,2015,Atlanta
2,1/5/2015,1.08,788025.06,137146.07,0.0,conventional,2015,BaltimoreWashington
3,1/5/2015,1.01,80034.32,6064.30,0.0,conventional,2015,Boise
4,1/5/2015,1.02,491738.00,87406.84,0.0,conventional,2015,Boston


In [18]:
#Gas In
#Gas CSV with ISO convention for Monday as Week Begin date
#Modified CSV before reading with Excel formula, including formatting
#(DateField)-WEEKDAY(DateField)+1 for Sunday begin date
#
Gas_file = "gas_prices.csv"
Gas_df = pd.read_csv(Gas_file)
Gas_df = Gas_df.rename(columns={"Date": "date", "Gasoline - All Grades": "gas_all_grades", "Regular": "regular", "Midgrade": "midgrade", "Premium": "premium", "Diesel (On-Highway) - All Types": "diesel"})
Gas_df = Gas_df[["date", "gas_all_grades", "regular", "midgrade", "premium", "diesel"]]
Gas_df.head(5000)


,date,gas_all_grades,regular,midgrade,premium,diesel
0,8/20/1990,NaN,1.191,NaN,NaN,NaN
1,8/27/1990,NaN,1.245,NaN,NaN,NaN
2,9/3/1990,NaN,1.242,NaN,NaN,NaN
3,9/10/1990,NaN,1.252,NaN,NaN,NaN
4,9/17/1990,NaN,1.266,NaN,NaN,NaN
...,...,...,...,...,...,...
1556,6/15/2020,2.185,2.098,2.495,2.745,2.403
1557,6/22/2020,2.216,2.129,2.521,2.773,2.425
1558,6/29/2020,2.260,2.174,2.563,2.812,2.430
1559,7/6/2020,2.265,2.177,2.571,2.821,2.437


In [20]:
# Avocado_df.head()
# Avocado_df2 = Avocado_df.loc[Avocado_df['region'] == 'TotalUS']
# Avocado_df2.head(3000)
# Gas_df.head(3000)

In [8]:
# XXXXXXXXXXXXXXXXX
# Connection to SQL
# XXXXXXXXXXXXXXXXX

In [21]:
#Enter your username substituted for USERNAME below; same for PASSWORD 
#Referencing pgAdmin4/PostgreSQL Username/Password
rds_connection_string = "USERNAME:PASSWORD@localhost:5432/ETL_Project"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [26]:
engine.table_names()

['gas', 'avocado']

In [27]:
Gas_df.to_sql(name='gas', con=engine, if_exists='append', index=False)

In [30]:
Avocado_df.to_sql(name='avocado', con=engine, if_exists='append', index=False)

In [31]:
pd.read_sql_query('select * from gas', con=engine).head()

,date,gas_all_grades,regular,midgrade,premium,diesel
0,8/20/1990,NaN,1.191,NaN,NaN,NaN
1,8/27/1990,NaN,1.245,NaN,NaN,NaN
2,9/3/1990,NaN,1.242,NaN,NaN,NaN
3,9/10/1990,NaN,1.252,NaN,NaN,NaN
4,9/17/1990,NaN,1.266,NaN,NaN,NaN


In [33]:
pd.read_sql_query('select * from avocado', con=engine).head()

,date,average_price,total_volume,small_avocados_sold,large_avocados_sold,xl_avocados_sold,type,year,region
0,1/5/2015,1.22,40873.28,9186.93,None,0.0,conventional,2015,Albany
1,1/5/2015,1.00,435021.49,16707.15,None,0.0,conventional,2015,Atlanta
2,1/5/2015,1.08,788025.06,137146.07,None,0.0,conventional,2015,BaltimoreWashington
3,1/5/2015,1.01,80034.32,6064.30,None,0.0,conventional,2015,Boise
4,1/5/2015,1.02,491738.00,87406.84,None,0.0,conventional,2015,Boston
